# Segmenting and Clustering Neighborhoods in Toronto-(Bourough:Scarborough)



In this analysis,I explored how to convert addresses into their equivalent latitude and longitude values. Also,learned how to use the Foursquare API to explore neighborhoods in Toronto City.I used the explore function to get the most common venue categories in each neighborhood, and then used this feature to group the neighborhoods into clusters.I used the k-means clustering algorithm to complete this task. Also used the Folium library to visualize the neighborhoods and their emerging clusters.

Table of Contents

1. Download and Explore Dataset
2. Explore Neighborhoods in Toronto ,Canada
3. Analyze Each Neighborhood
4. Cluster Neighborhoods
5. Examine Clusters

In [1]:
#Downloading the dependencies 

from bs4 import BeautifulSoup #library for web scraping
import requests  # library to handle requests
import json  # library to handle JSON files
import xml
import pandas as pd #Python library data manipulation and analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text

soup = BeautifulSoup(source, 'lxml')#Beautiful Soup to Parse the url page

table=soup.find('table') #Finds the required table area

#List initialization to collect the Postalcodes, Boroughs and Neighborhoods
postalcode=[]
borough=[]
neighborhood=[]

x=table.tbody #Navidation to the body of the table

for tr in x.find_all('tr'): #Iterating through rows and columns
    td=tr.find_all('td')
    for x in td:
        #Exception handling to bypass the errors
        try:
            if x.span.text!='Not assigned':
                postalcode.append(x.b.text) #Collecting the Postalcodes
        except:
            pass
        try:
            if x.span.text!='Not assigned': 
                my=x.span.get_text(separator=u' ').split(' ')
                
                if my[1]=='York':
                    my_b= my[0]+' '+my[1]
                    borough.append(my_b)
                    my_nei=my[2:]
                    if my_nei==['\n'] or my_nei==[]:
                        neighborhood.append(my_b)
                    else:
                        my_nei1=' '.join(my_nei)
                        neighborhood.append(my_nei1.replace('(',',',5).replace(')',',',5).replace('/',',',5).strip(', '))
                
                elif my[1]=='Toronto':
                    my_b=my[0]+' '+my[1]
                    borough.append(my_b)
                    my_nei=my[2:]
                    if my_nei==['\n'] or my_nei==[]:
                        neighborhood.append(my_b)
                    else:
                        my_nei1=' '.join(my_nei)
                        neighborhood.append(my_nei1.replace('(',',',5).replace(')',',',5).replace('/',',',5).strip(', '))
                
                elif my[1]=='Park':
                    my_b=my[0]+' '+my[1]
                    borough.append(my_b)
                    my_nei=my[2:]
                    if my_nei==['\n'] or my_nei==[]:
                        neighborhood.append(my_b)
                    else:
                        my_nei1=' '.join(my_nei)
                        neighborhood.append(my_nei1.replace('(',',',5).replace(')',',',5).replace('/',',',5).strip(', '))
                
                else:
                    my_b=my[0]
                    borough.append(my_b)
                    my_nei=my[2:]
                    if my_nei==['\n'] or my_nei==[]:
                        neighborhood.append(my_b)
                    else:
                        my_nei1=' '.join(my_nei)
                        neighborhood.append(my_nei1.replace('(',',',5).replace(')',',',5).replace('/',',',5).strip(', '))
       
    
         
        except:
            pass

In [3]:
#Constructing the main dataframe 
main_df=pd.DataFrame({'PostalCode':postalcode,
                 'Borough':borough,
                 'Neighborhood':neighborhood})

In [4]:
#Fixing the order of the columns in the main dataframe
main_df=main_df[['PostalCode','Borough','Neighborhood']]

In [5]:
#Visualizing the first few rows of the dataframe
main_df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern , Rouge"
7,M3B,North York,"Don Mills , North"
8,M4B,East York,"Parkview Hill , Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District , Ryerson"


In [6]:
len(main_df['PostalCode'])

103

In [9]:
# The code was removed by Watson Studio for sharing.

In [ ]:
latitude=[] #List to collect the latitudes
longitude=[] #List to collect the longitudes

for i in main_df['PostalCode']: #Iterating through Postalcodes to collect the locations data
    j='toronto,'+i
    try:
        url ="https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}".format(API_key,j)
        
        response = requests.get(url).json() # get response
        geographical_data = response['results'][0]['geometry']['location'] # get geographical coordinates
        
        latitude.append(geographical_data['lat'])
        longitude.append(geographical_data['lng'])
    except:
        pass


In [10]:
len(latitude)

103

In [11]:
main_df['Latitude']=latitude #Adding a column in the main dataframe for Latitude  

main_df['Longitude']=longitude #Adding a column in the main dataframe for Longitude 
main_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [12]:
main_df.Borough.unique()

array(['North York', 'Downtown Toronto', "Queen's Park", 'Etobicoke',
       'Scarborough', 'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

## Use geopy library to get the latitude and longitude values of Toronto

In [13]:
address = 'Toronto,Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude_x = location.latitude
longitude_y = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_x, longitude_y))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


## Create a map of Toronto with neighborhoods superimposed on top.

In [14]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude_x, longitude_y], zoom_start=10)

# add markers to map
for lat, lng, bor, nei in zip(main_df['Latitude'], main_df['Longitude'], main_df['Borough'], main_df['Neighborhood']):
    
    label = '{}, {}'.format(nei, bor)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [15]:
Scarborough_data = main_df[main_df['Borough'] == 'Scarborough'].reset_index(drop=True)
Scarborough_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park , Ionview , East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile , Clairlea , Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside , Cliffcrest , Scarborough Villa...",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff , Cliffside West",43.692657,-79.264848


## Geographical coordinates of Scarborough

In [16]:
address = 'Scarborough,Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude_s = location.latitude
longitude_s = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude_s, longitude_s))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


#### Neighborhoods in Scarborough

In [17]:
# create map of Scarborough using latitude and longitude values
map_scarborough = folium.Map(location=[latitude_s, longitude_s], zoom_start=10)

# add markers to map
for lat, lng, bor, nei in zip(Scarborough_data['Latitude'], Scarborough_data['Longitude'],Scarborough_data['Borough'], Scarborough_data['Neighborhood']):
    
    label = '{}, {}'.format(nei, bor)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

## Foursquare Credentials and Version

In [50]:
# The code was removed by Watson Studio for sharing.

### Exploring the first neighborhood of Scarborough

In [19]:
#Name of the neighborhood
first_nei=Scarborough_data['Neighborhood'][0]
first_nei

'Malvern  ,  Rouge'

In [20]:
#Location of the neighborhood
first_nei_lat=Scarborough_data.loc[0,'Latitude']
first_nei_lon=Scarborough_data.loc[0,'Longitude']
print('Latitude and longitude values of {} are {}, {}.'.format(first_nei, 
                                                               first_nei_lat, 
                                                               first_nei_lon))

Latitude and longitude values of Malvern  ,  Rouge are 43.8066863, -79.1943534.


Now, let's get the top 100 venues that are in Malvern within a radius of 500 meters.

In [51]:
radius = 500 
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    first_nei_lat, 
    first_nei_lon, 
    radius, 
    LIMIT)
 

In [22]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5b6ca41c351e3d1d39db5c1b'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4bb6b9446edc76b0d771311c-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/fastfood_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d16e941735',
         'name': 'Fast Food Restaurant',
         'pluralName': 'Fast Food Restaurants',
         'primary': True,
         'shortName': 'Fast Food'}],
       'id': '4bb6b9446edc76b0d771311c',
       'location': {'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Morningside & Sheppard',
        'distance': 387,
        'formattedAddress': ['Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.80744841934756,
          'ln

#### Since we havent received much results in this neighborhood lets select a different neighborhood 


In [23]:
second_nei_name=Scarborough_data.loc[15,'Neighborhood']
second_nei_lat=Scarborough_data.loc[15,'Latitude']
second_nei_lon=Scarborough_data.loc[15,'Longitude']
print('Latitude and longitude values of {} are {}, {}.'.format(second_nei_name, 
                                                               second_nei_lat, 
                                                               second_nei_lon))

Latitude and longitude values of Steeles  West ,  L'Amoreaux  West are 43.7995252, -79.3183887.


In [52]:
radius = 500 
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    second_nei_lat, 
    second_nei_lon, 
    radius, 
    LIMIT)


In [25]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5b6ca41e351e3d1d3a40734c'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4df28ea0e4cda09e6da0129a-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d145941735',
         'name': 'Chinese Restaurant',
         'pluralName': 'Chinese Restaurants',
         'primary': True,
         'shortName': 'Chinese'}],
       'id': '4df28ea0e4cda09e6da0129a',
       'location': {'address': '2900 Warden Ave',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'at Finch Ave. E',
        'distance': 72,
        'formattedAddress': ['2900 Warden Ave (at Finch Ave. E)',
         'Toronto ON M1W 2S8',
         'Canada'],
        'labe

In [26]:
results['response']['groups'][0]['items'][0]['venue']['categories'][0]['name']

'Chinese Restaurant'

### All the information is in the items key. Lets create a get_category_type function .

In [27]:
venues=results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
nearby_venues.columns

Index(['reasons.count', 'reasons.items', 'referralId', 'venue.categories',
       'venue.id', 'venue.location.address', 'venue.location.cc',
       'venue.location.city', 'venue.location.country',
       'venue.location.crossStreet', 'venue.location.distance',
       'venue.location.formattedAddress', 'venue.location.labeledLatLngs',
       'venue.location.lat', 'venue.location.lng',
       'venue.location.neighborhood', 'venue.location.postalCode',
       'venue.location.state', 'venue.name', 'venue.photos.count',
       'venue.photos.groups'],
      dtype='object')

In [28]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [29]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Mr Congee Chinese Cuisine 龍粥記,"[{'id': '4bf58dd8d48988d145941735', 'shortName...",43.798879,-79.318335
1,Phoenix Restaurant 金鳳餐廳,"[{'id': '4bf58dd8d48988d145941735', 'shortName...",43.798198,-79.318432
2,Subway,"[{'id': '4bf58dd8d48988d1c5941735', 'shortName...",43.798983,-79.318838
3,Price Chopper,"[{'id': '4bf58dd8d48988d118951735', 'shortName...",43.799445,-79.318563
4,Tim Hortons,"[{'id': '4bf58dd8d48988d1e0931735', 'shortName...",43.798281,-79.318317
5,Shoppers Drug Mart,"[{'id': '4bf58dd8d48988d10f951735', 'shortName...",43.799670,-79.319315
6,KFC,"[{'id': '4bf58dd8d48988d16e941735', 'shortName...",43.798938,-79.318854
7,Yamamoto Japanese Cuisine 山本盛世,"[{'id': '4bf58dd8d48988d111941735', 'shortName...",43.798589,-79.318558
8,McDonald's,"[{'id': '4bf58dd8d48988d16e941735', 'shortName...",43.798880,-79.318724
9,Pizza Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'shortName...",43.797909,-79.318113


In [30]:
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Mr Congee Chinese Cuisine 龍粥記,Chinese Restaurant,43.798879,-79.318335
1,Phoenix Restaurant 金鳳餐廳,Chinese Restaurant,43.798198,-79.318432
2,Subway,Sandwich Place,43.798983,-79.318838
3,Price Chopper,Grocery Store,43.799445,-79.318563
4,Tim Hortons,Coffee Shop,43.798281,-79.318317
5,Shoppers Drug Mart,Pharmacy,43.799670,-79.319315
6,KFC,Fast Food Restaurant,43.798938,-79.318854
7,Yamamoto Japanese Cuisine 山本盛世,Japanese Restaurant,43.798589,-79.318558
8,McDonald's,Fast Food Restaurant,43.798880,-79.318724
9,Pizza Pizza,Pizza Place,43.797909,-79.318113


# Explore Neighborhoods in Scarborough
### Let's create a function to repeat the same process to all the neighborhoods in Scarborough

In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        venue_results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in venue_results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Following script run the above function on each neighborhood and create a new dataframe called scarborough_venues.

In [32]:
scarborough_venues = getNearbyVenues(names=Scarborough_data['Neighborhood'],
                                   latitudes=Scarborough_data['Latitude'],
                                   longitudes=Scarborough_data['Longitude']
                                  )



Malvern  ,  Rouge
Rouge Hill  ,  Port Union  ,  Highland Creek
Guildwood  ,  Morningside  ,  West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park  ,  Ionview  , East  Birchmount Park
Golden Mile  ,  Clairlea  ,  Oakridge
Cliffside  ,  Cliffcrest  ,  Scarborough Village  West
Birch Cliff  ,  Cliffside  West
Dorset Park  ,  Wexford Heights  ,  Scarborough Town Centre
Wexford  ,  Maryvale
Agincourt
Corners ,  Tam O'Shanter  ,  Sullivan
Milliken  ,  Agincourt North  ,  Steeles  East ,  L'Amoreaux  East
Steeles  West ,  L'Amoreaux  West
Upper Rouge


In [33]:
print(scarborough_venues.shape)
scarborough_venues.head()

(90, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern , Rouge",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
3,"Guildwood , Morningside , West Hill",43.763573,-79.188711,Swiss Chalet,43.767697,-79.189914,BBQ Joint
4,"Guildwood , Morningside , West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


### Venues per neighborhood

In [34]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Birch Cliff , Cliffside West",4,4,4,4,4,4
Cedarbrae,7,7,7,7,7,7
"Cliffside , Cliffcrest , Scarborough Village West",2,2,2,2,2,2
"Corners , Tam O'Shanter , Sullivan",9,9,9,9,9,9
"Dorset Park , Wexford Heights , Scarborough Town Centre",7,7,7,7,7,7
"Golden Mile , Clairlea , Oakridge",10,10,10,10,10,10
"Guildwood , Morningside , West Hill",6,6,6,6,6,6
"Kennedy Park , Ionview , East Birchmount Park",7,7,7,7,7,7


### Unique Categories

In [35]:

print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 56 uniques categories.


# Analyze Each Neighborhood

In [36]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]
scarborough_onehot.head()


,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Cosmetics Shop,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,General Entertainment,Grocery Store,Hakka Restaurant,Health & Beauty Service,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Moving Target,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Train Station,Vietnamese Restaurant
0,"Malvern , Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill , Port Union , Highland Creek",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Rouge Hill , Port Union , Highland Creek",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood , Morningside , West Hill",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood , Morningside , West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [37]:
scarborough_onehot.shape   #90 venues and 56 unique categories

(90, 57)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category¶

In [38]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Cosmetics Shop,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,General Entertainment,Grocery Store,Hakka Restaurant,Health & Beauty Service,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Moving Target,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Train Station,Vietnamese Restaurant
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.200000,0.0,0.0,0.00,0.000000,0.200000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.2,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.20,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
1,"Birch Cliff , Cliffside West",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.25,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
2,Cedarbrae,0.000000,0.142857,0.000000,0.000000,0.142857,0.142857,0.0,0.000000,0.0,0.0,0.00,0.142857,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.00,0.000000,0.142857,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.142857,0.000000,0.000000,0.000000
3,"Cliffside , Cliffcrest , Scarborough Villa...",0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.5,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
4,"Corners , Tam O'Shanter , Sullivan",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000000,0.111111,0.111111,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.111111,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.111111,0.0,0.000000,0.000000,0.222222,0.000000,0.000000,0.000000,0.111111,0.00,0.000000,0.0,0.111111,0.000000,0.000000,0.000000
5,"Dorset Park , Wexford Heights , Scarboroug...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000000,0.142857,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.285714,0.0,0.000000,0.000000,0.00,0.142857,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.142857,0.000000,0.142857
6,"Golden Mile , Clairlea , Oakridge",0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.0,0.000000,0.2,0.1,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.1,0.000000,0.1,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.1,0.000000,0.000000,0.0,0.0,0.000000,0.1,0.000000,0.000000,0.000000,0

In [39]:
scarborough_grouped.shape

(16, 57)

### Each neighborhood along with the top 5 most common venues

In [40]:
num_top_venues = 5
for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp =scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                venue  freq
0              Lounge   0.2
1      Breakfast Spot   0.2
2        Skating Rink   0.2
3      Sandwich Place   0.2
4  Chinese Restaurant   0.2


----Birch Cliff  ,  Cliffside  West----
                   venue  freq
0        College Stadium  0.25
1           Skating Rink  0.25
2  General Entertainment  0.25
3                   Café  0.25
4              Pet Store  0.00


----Cedarbrae----
                  venue  freq
0  Caribbean Restaurant  0.14
1    Athletics & Sports  0.14
2                Bakery  0.14
3                  Bank  0.14
4       Thai Restaurant  0.14


----Cliffside  ,  Cliffcrest  ,  Scarborough Village  West----
                 venue  freq
0  American Restaurant   0.5
1                Motel   0.5
2   Athletics & Sports   0.0
3  Japanese Restaurant   0.0
4    Korean Restaurant   0.0


----Corners ,  Tam O'Shanter  ,  Sullivan----
                venue  freq
0         Pizza Place  0.22
1  Italian Restaurant  0.11
2        Noodle

### Function to sort the venues in descending order.

In [41]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [42]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Sandwich Place,Lounge,Breakfast Spot,Chinese Restaurant,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Fried Chicken Joint
1,"Birch Cliff , Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,Fried Chicken Joint,Fast Food Restaurant
2,Cedarbrae,Athletics & Sports,Thai Restaurant,Hakka Restaurant,Bakery,Bank,Fried Chicken Joint,Caribbean Restaurant,Vietnamese Restaurant,Convenience Store,Grocery Store
3,"Cliffside , Cliffcrest , Scarborough Villa...",American Restaurant,Motel,Ice Cream Shop,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,"Corners , Tam O'Shanter , Sullivan",Pizza Place,Noodle House,Chinese Restaurant,Thai Restaurant,Fried Chicken Joint,Italian Restaurant,Shopping Mall,Coffee Shop,General Entertainment,Fast Food Restaurant
5,"Dorset Park , Wexford Heights , Scarboroug...",Indian Restaurant,Pet Store,Chinese Restaurant,Latin American Restaurant,Vietnamese Restaurant,Thrift / Vintage Store,Smoke Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
6,"Golden Mile , Clairlea , Oakridge",Bus Line,Bakery,Ice Cream Shop,Metro Station,Intersection,Soccer Field,Park,Bus Station,Department Store,Discount Store
7,"Guildwood , Morningside , West Hill",BBQ Joint,Electronics Store,Breakfast Spot,Rental Car Location,Medical Center,Mexican Restaurant,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment
8,"Kennedy Park , Ionview , East Birchmount Park",Discount Store,Coffee Shop,Department Store,Convenience Store,Chinese Restaurant,Train Station,College Stadium,Health & Beauty Service,Hakka Restaurant,Grocery Store
9,"Malvern , Rouge",Fast Food Restaurant,Vietnamese Restaurant,Train Station,Health & Beauty Service,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store


# Cluster Neighborhoods

In [43]:
# set number of clusters
# Using k-means to cluster the neighborhood into 4 clusters.
kclusters = 4

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 1, 1, 3, 1, 1, 1, 1, 1, 2, 0, 1, 0, 1, 1, 1], dtype=int32)

Creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

Since the neighborhood 'Upper Rouge' has no venue data returned lets remove that neighborhood

In [44]:
scarborough_merged = Scarborough_data[0:16]

# add clustering labels
scarborough_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scarborough_merged.head() # check the last columns!

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353,1,Fast Food Restaurant,Vietnamese Restaurant,Train Station,Health & Beauty Service,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497,1,Bar,Moving Target,Vietnamese Restaurant,College Stadium,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711,1,BBQ Joint,Electronics Store,Breakfast Spot,Rental Car Location,Medical Center,Mexican Restaurant,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment
3,M1G,Scarborough,Woburn,43.770992,-79.216917,3,Coffee Shop,Korean Restaurant,Indian Restaurant,Auto Garage,Convenience Store,Hakka Restaurant,Grocery Store,Athletics & Sports,General Entertainment,Fried Chicken Joint
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,Athletics & Sports,Thai Restaurant,Hakka Restaurant,Bakery,Bank,Fried Chicken Joint,Caribbean Restaurant,Vietnamese Restaurant,Convenience Store,Grocery Store


### Visualising the clusters

In [45]:
# create map
map_clusters = folium.Map(location=[latitude_s, longitude_s], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
colors_array = cm.rainbow(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)
# add markers to the map
markers_colors = []
for lat, lon, nei , cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighborhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(nei) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

['#8000ff', '#2adddd', '#d4dd80', '#ff0000']


## Examine Clusters
#### Examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we can then assign a name to each cluster.



#### Cluster 1

In [46]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0,scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,"Dorset Park , Wexford Heights , Scarboroug...",0,Indian Restaurant,Pet Store,Chinese Restaurant,Latin American Restaurant,Vietnamese Restaurant,Thrift / Vintage Store,Smoke Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
12,Agincourt,0,Skating Rink,Sandwich Place,Lounge,Breakfast Spot,Chinese Restaurant,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Fried Chicken Joint


#### Cluster 2

In [47]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1,scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Malvern , Rouge",1,Fast Food Restaurant,Vietnamese Restaurant,Train Station,Health & Beauty Service,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store
1,"Rouge Hill , Port Union , Highland Creek",1,Bar,Moving Target,Vietnamese Restaurant,College Stadium,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,"Guildwood , Morningside , West Hill",1,BBQ Joint,Electronics Store,Breakfast Spot,Rental Car Location,Medical Center,Mexican Restaurant,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment
4,Cedarbrae,1,Athletics & Sports,Thai Restaurant,Hakka Restaurant,Bakery,Bank,Fried Chicken Joint,Caribbean Restaurant,Vietnamese Restaurant,Convenience Store,Grocery Store
5,Scarborough Village,1,Health & Beauty Service,Grocery Store,Playground,Coffee Shop,Hakka Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
6,"Kennedy Park , Ionview , East Birchmount Park",1,Discount Store,Coffee Shop,Department Store,Convenience Store,Chinese Restaurant,Train Station,College Stadium,Health & Beauty Service,Hakka Restaurant,Grocery Store
7,"Golden Mile , Clairlea , Oakridge",1,Bus Line,Bakery,Ice Cream Shop,Metro Station,Intersection,Soccer Field,Park,Bus Station,Department Store,Discount Store
8,"Cliffside , Cliffcrest , Scarborough Villa...",1,American Restaurant,Motel,Ice Cream Shop,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
11,"Wexford , Maryvale",1,Sandwich Place,Breakfast Spot,Auto Garage,Smoke Shop,Shopping Mall,Middle Eastern Restaurant,Convenience Store,Cosmetics Shop,Department Store,Discount Store
13,"Corners , Tam O'Shanter , Sullivan",1,Pizza Place,Noodle House,Chinese Restaurant,Thai Restaurant,Fried Chicken Joint,Italian Restaurant,Shopping Mall,Coffee Shop,General Entertainment,Fast Food Restaurant


#### Cluster 3

In [48]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2,scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,"Birch Cliff , Cliffside West",2,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,Fried Chicken Joint,Fast Food Restaurant


#### Cluster 4

In [49]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3,scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Woburn,3,Coffee Shop,Korean Restaurant,Indian Restaurant,Auto Garage,Convenience Store,Hakka Restaurant,Grocery Store,Athletics & Sports,General Entertainment,Fried Chicken Joint


  <div align="center">
    <H1> Thank You!! </H1>
</div> 